In [2]:
from ultralytics import YOLO
model10n = YOLO('trained_weights/v10n.pt')
model10n.export(format="openvino", imgsz=480)
model10s = YOLO('trained_weights/v10s.pt')
model10s.export(format="openvino", imgsz=480)

Ultralytics YOLOv8.2.73 🚀 Python-3.9.19 torch-2.0.1+cu117 CPU (12th Gen Intel Core(TM) i9-12900H)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs

PyTorch: starting from 'trained_weights/v10n.pt' with input shape (1, 3, 480, 480) BCHW and output shape(s) (1, 300, 6) (5.5 MB)

OpenVINO: starting export with openvino 2024.2.0-15519-5c0f38f83f6-releases/2024/2...
OpenVINO: export success ✅ 3.2s, saved as 'trained_weights/v10n_openvino_model/' (9.0 MB)

Export complete (4.7s)
Results saved to /home/kenna/dvd/trained_weights
Predict:         yolo predict task=detect model=trained_weights/v10n_openvino_model imgsz=480  
Validate:        yolo val task=detect model=trained_weights/v10n_openvino_model imgsz=480 data=data.yaml  
Visualize:       https://netron.app
Ultralytics YOLOv8.2.73 🚀 Python-3.9.19 torch-2.0.1+cu117 CPU (12th Gen Intel Core(TM) i9-12900H)
YOLOv10s summary (fused): 293 layers, 8,036,508 parameters, 0 gradients, 24.4 GFLOPs

PyTorch: starti

'trained_weights/v10s_openvino_model'

In [3]:
model10n.export(format='onnx', imgsz=480)
model10s.export(format='onnx', imgsz=480)

Ultralytics YOLOv8.2.73 🚀 Python-3.9.19 torch-2.0.1+cu117 CPU (12th Gen Intel Core(TM) i9-12900H)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs

PyTorch: starting from 'trained_weights/v10n.pt' with input shape (1, 3, 480, 480) BCHW and output shape(s) (1, 300, 6) (5.5 MB)

ONNX: starting export with onnx 1.14.0 opset 17...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: export success ✅ 0.5s, saved as 'trained_weights/v10n.onnx' (8.8 MB)

Export complete (1.8s)
Results saved to /home/kenna/dvd/trained_weights
Predict:         yolo predict task=detect model=trained_weights/v10n.onnx imgsz=480  
Validate:        yolo val task=detect model=trained_weights/v10n.onnx imgsz=480 data=data.yaml  
Visualize:       https://netron.app
Ultralytics YOLOv8.2.73 🚀 Python-3.9.19 torch-2.0.1+cu117 CPU (12th

'trained_weights/v10s.onnx'

In [19]:
import nncf
import openvino.runtime as ov
import torch
from torchvision import datasets, transforms
import openvino as ov

# Instantiate your uncompressed model
model10n = ov.Core().read_model("trained_weights/v10n.onnx")
model10s = ov.Core().read_model("trained_weights/v10s.onnx")

# Provide validation part of the dataset to collect statistics needed for the compression algorithm
val_dataset = datasets.ImageFolder("train_val_data/", transform=transforms.Compose([transforms.RandomResizedCrop(480),transforms.ToTensor()]))
dataset_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1)

# Step 1: Initialize transformation function
def transform_fn(data_item):
    images, _ = data_item
    return images

# Step 2: Initialize NNCF Dataset
calibration_dataset = nncf.Dataset(dataset_loader, transform_fn)
# Step 3: Run the quantization pipeline
quantized_model10n = nncf.quantize(model10n, calibration_dataset)
ov.save_model(quantized_model10n, 'trained_weights/v10n_quantized_openvino_model/v10n_quantized.xml')

quantized_model_10s = nncf.quantize(model10s, calibration_dataset)
ov.save_model(quantized_model_10s, 'trained_weights/v10s_quantized_openvino_model/v10s_quantized.xml')

In [20]:
drone_cpu = ov.Core().read_model("trained_weights/drone_cpu.onnx")
quantized_drone_cpu = nncf.quantize(drone_cpu, calibration_dataset)
ov.save_model(quantized_drone_cpu, 'trained_weights/drone_cpu_quantized_openvino_model/drone_cpu_quantized.xml')


In [21]:
# import openvino as ov
# model = ov.convert_model('model_weights/yolov5v2n/best.onnx')
# ov.save_model(model, 'eagle_model_weights/yolov5v2n.xml')